# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [5]:
from geopy.distance import geodesic
import numpy as np
import pandas as pd
import gurobipy as gp
import folium

# Data
data = pd.read_csv('chengdu-spot-89.csv')

n = data.shape[0]
names = data['name'].values
categories = data['category'].values
lat_vals = data['lat'].values
lng_vals = data['lng'].values

# 构建坐标字典（根据 name 索引）
name_to_index = {name: i for i, name in enumerate(names)}

# 构建(lat, lng)坐标对列表
coordinates = list(zip(lat_vals, lng_vals))
name_to_coord = dict(zip(names, coordinates))

# 优化后的马拉松赛事评分标准
category_score_map = {
    '地标': 10,
    '展览馆': 9,
    '广场': 8.5,
    '城市公园': 8.5,
    '博物馆': 8,
    '文化公园': 7.5,
    '森林公园': 7.5,
    '自然风光': 7,
    '休闲公园': 6.5,
    '休闲街区': 6,
    '美术馆': 6,
    '植物园': 5.5,
    '历史建筑': 5,
    '宗教': 4,
    '人文古迹': 4,
    '主题公园': 3.5,
    '动物园': 3,
    '海洋馆': 2.5
}
data['score'] = data['category'].map(category_score_map) # 根据category打分

TPO10 = ['宽窄巷子', '环球中心', '成都大熊猫繁育研究基地', '双子塔', '成都杜甫草堂博物馆', '天府广场', '成都武侯祠博物馆', '成都太古里', '青羊宫', '成都339']  # 前十名景点
data.loc[data['name'].isin(TPO10), 'score'] += 8 # 前十名景点额外加分
scores = data['score'].values


# model
# 定义曼哈顿距离函数
def manhattan_distance(p1, p2):
    lat1, lng1 = p1
    lat2, lng2 = p2
    # 南北方向距离
    d_lat = geodesic((lat1, lng1), (lat2, lng1)).meters
    # 东西方向距离
    d_lng = geodesic((lat1, lng1), (lat1, lng2)).meters
    return d_lat + d_lng

# 构建曼哈顿距离矩阵
def build_manhattan_distance_matrix(coords):
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            d = manhattan_distance(coords[i], coords[j])
            d = round(d, 4)  # 保留4位小数
            dist_matrix[i][j] = d
            dist_matrix[j][i] = d
    return dist_matrix

distance_matrix = build_manhattan_distance_matrix(coordinates)

# 起点和终点索引
start_name = '金沙遗址博物馆'
end_name = '成都新国际会展中心'
start_index = name_to_index[start_name]
end_index = name_to_index[end_name]

# 初始化模型
m = gp.Model("MarathonPathOptimization")

# 决策变量
x = m.addVars(n, vtype=gp.GRB.BINARY, name="x")  # 是否选中点
y = m.addVars(n, n, vtype=gp.GRB.BINARY, name="y")  # 边
for i in range(n):
    y[i,i].ub = 0 # no self-loops

# 目标函数：最大化得分
m.setObjective(gp.quicksum(scores[i] * x[i] for i in range(n)), gp.GRB.MAXIMIZE)

# 限制
# 距离约束
max_distance = 42237.195 # 马拉松全程42.195公里，允许有误差(0.1%)
m.addConstr(gp.quicksum(distance_matrix[i][j] * y[i, j] for i in range(n) for j in range(n)) <= max_distance)

# 每个非起终点：有入就有出
for i in range(n):
    if i != start_index and i != end_index:
        m.addConstr(gp.quicksum(y[j, i] for j in range(n)) == x[i])
        m.addConstr(gp.quicksum(y[i, j] for j in range(n)) == x[i])

# 起点出发
m.addConstr(gp.quicksum(y[start_index, j] for j in range(n)) == 1)
m.addConstr(gp.quicksum(y[i, start_index] for i in range(n)) == 0)

# 终点到达
m.addConstr(gp.quicksum(y[i, end_index] for i in range(n)) == 1)
m.addConstr(gp.quicksum(y[end_index, j] for j in range(n)) == 0)

# 起终点必须选择
m.addConstr(x[start_index] == 1)
m.addConstr(x[end_index] == 1)

# MTZ 防止子回路（subtour）
u = m.addVars(n, lb=0, ub=n - 1, vtype=gp.GRB.INTEGER, name="u") # 顺序变量 u[i]，表示第 i 个点在路径中的访问顺序
m.addConstr(u[start_index] == 0) # 起点的访问顺序设为0（起点为 0）
m.addConstr(u[end_index] == n-1) # 终点的访问顺序设为 n-1（终点为 n-1）

# 只对非起点、非终点点之间添加MTZ约束（因为这里是开路不是环）
for i in range(n):
    for j in range(n):
        if i != j and i != start_index and i != end_index and j != start_index and j != end_index:
            m.addConstr(u[i] - u[j] + n * y[i, j] <= n - 1)


# solve
m.optimize()

# 输出结果
# 提取路径中的边（即 y[i,j] == 1）
edges = []
if m.status == gp.GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:
                edges.append((i, j))

    # 重建路径顺序
    path = [start_index]
    current = start_index
    while current != end_index:
        for i, j in edges:
            if i == current:
                path.append(j)
                current = j
                break

    print("最终路径经过的景点：")
    for idx in path:
        print(f" - {names[idx]} (Score: {scores[idx]})")

    # 可视化路径
    # 创建地图对象
    start_coord = coordinates[start_index]
    route_map = folium.Map(location=start_coord, zoom_start=12)

    # 添加景点标记和路径线
    for i, idx in enumerate(path):
        lat, lng = coordinates[idx]
        popup_text = f"{i+1}. {names[idx]} (score: {scores[idx]})"
        folium.Marker(
            location=(lat, lng),
            popup=popup_text,
            tooltip=popup_text,
            icon=folium.Icon(color='blue' if idx not in [start_index, end_index] else 'green' if idx == start_index else 'red')
        ).add_to(route_map)

    # 画路径线
    path_coords = [coordinates[i] for i in path]
    folium.PolyLine(path_coords, color="blue", weight=4, opacity=0.7).add_to(route_map)

    # 保存到 HTML 文件
    route_map.save("marathon_route5.html")
    print("路径已保存到 marathon_route5.html")

    # 输出路径长度和得分
    total_distance = sum(distance_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1))
    total_score = sum(scores[idx] for idx in path)
    print(f"总路径长度: {total_distance:.4f} 米")
    print(f"总得分: {total_score:.2f}")

    # 输出路径
    print("路径节点索引：", path)
else:
    print("优化失败，状态码：", m.status)

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 7665 rows, 8099 columns and 46298 nonzeros
Model fingerprint: 0x05539e59
Variable types: 0 continuous, 8099 integer (8010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+04]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 9e+01]
  RHS range        [1e+00, 4e+04]
Found heuristic solution: objective 17.0000000
Presolve removed 226 rows and 489 columns
Presolve time: 0.05s
Presolved: 7439 rows, 7610 columns, 44267 nonzeros
Variable types: 0 continuous, 7610 integer (7523 binary)

Root relaxation: objective 3.950811e+02, 2046 iterations, 0.04 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  395.08110    0   72   17.

In [6]:
import requests
import folium
import pandas as pd
import time

# Data
path = [36, 34, 59, 33, 64, 72, 38, 58, 31, 20, 22, 71, 32, 70, 21, 61, 43, 13, 1, 29, 56, 55, 28, 51, 42, 41, 40, 50, 25, 48, 49, 68] # 优化后的路径索引
data = pd.read_csv('chengdu-spot-89.csv')
names = data['name'].values
lat_vals = data['lat'].values
lng_vals = data['lng'].values

# 设置高德 Web API Key
GAODE_KEY = '43431d11216b1ba36af9d8556934046d'

# 设置路径函数
def get_gaode_route(lng1, lat1, lng2, lat2, key):
    url = (
        f"https://restapi.amap.com/v3/direction/walking?"
        f"key={key}&origin={lng1},{lat1}&destination={lng2},{lat2}"
    )
    response = requests.get(url)
    data = response.json()
    if data["status"] == "1" and data["route"]["paths"]:
        steps = data["route"]["paths"][0]["steps"]
        route_coords = []
        for step in steps:
            for point in step["polyline"].split(";"):
                lng, lat = map(float, point.split(","))
                route_coords.append((lat, lng)) # 高德 API 返回的坐标是 (经度, 纬度)，需要转换为 (纬度, 经度)
                distance = float(data["route"]["paths"][0]["distance"])  # 单位：米
        return route_coords, distance
    else:
        print(f"请求失败: {data.get('info', 'unknown error')}")
        return [], 0.0

# 获取路径坐标
total_real_distance = 0.0
full_path_coords = []

for i in range(len(path) - 1):
    idx_start = path[i]
    idx_end = path[i + 1]

    lng1, lat1 = lng_vals[idx_start], lat_vals[idx_start]
    lng2, lat2 = lng_vals[idx_end], lat_vals[idx_end]

    segment_coords, segment_distance = get_gaode_route(lng1, lat1, lng2, lat2, key=GAODE_KEY)

    if not segment_coords:
        continue

    if full_path_coords and full_path_coords[-1] == segment_coords[0]:
        segment_coords = segment_coords[1:]

    full_path_coords.extend(segment_coords)
    total_real_distance += segment_distance

    time.sleep(0.3)

print(f"高德导航返回的现实可通行路径长度：{total_real_distance:.4f} 米")

# 创建地图对象
start_idx = path[0]
start_lat, start_lng = lat_vals[start_idx], lng_vals[start_idx]
route_map = folium.Map(location=[start_lat, start_lng], zoom_start=12)

# 添加路线
folium.PolyLine(full_path_coords, color='blue', weight=4, opacity=0.7).add_to(route_map)

# 添加节点标记
for idx in path:
    lat, lng = lat_vals[idx], lng_vals[idx]
    folium.Marker([lat, lng], popup=names[idx], tooltip=names[idx],
                  icon=folium.Icon(color='green' if idx == path[0]
                                   else 'red' if idx == path[-1]
                                   else 'blue')).add_to(route_map)

# 保存结果
route_map.save("realistic_marathon_route.html")
print("已保存为 realistic_marathon_route.html")

高德导航返回的现实可通行路径长度：48283.0000 米
已保存为 realistic_marathon_route.html
